# Imports

In [11]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from collections import defaultdict
import torch
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from dataclasses import dataclass

In [12]:
from utils.data_retrieval import process_annotations
from utils.categorisation import categorize_facial_mot, categorize_utterance_mot, categorize_gaze_mot, categorize_gaze_chi
from utils.multimodal_dataset import MultimodalDataset
from utils.dataset_split import DatasetSplit
from utils.train_loop import train_loop
from utils.evaluation_loop import evaluation_loop
from utils.helper import get_quality_metrics
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [14]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Data Preparation

In [15]:
file_name = "A_12_20130925_1212_01_2.txt"

In [16]:
process_annotations(file_name, "annotations_per_t.csv")

Per-second annotations saved to /content/drive/MyDrive/ISIR/RedBallGame/final/annotations_per_t.csv


## Annotations categorization

In [17]:
df = pd.read_csv("annotations_per_t.csv")

### Action@MOT

In [18]:
action_mot_counts = df[df['Tier']=='Action@MOT'].value_counts('Annotation')
print(action_mot_counts)

Annotation
holding the ball in mouth                       271
picking up the ball                             186
building a toy pyramid                          153
moving towards the ball to take it              111
putting the ball in mouth                       103
tossing the ball                                 84
taking the ball with mouth                       69
showing the glass                                51
extending a hand                                 51
spitting out the ball                            50
attracting dad's attention to the game           46
spitting the ball into the glass                 34
holding the glass                                28
taking the ball                                  22
tossing the toy                                  21
throwing the ball at the child                   18
picking up the glass                             16
tickling the child                               16
shaking the glass with the ball                  16
t

In [19]:
def standardize_action_annotations(df: pd.DataFrame):
    df = df.copy() # Work on a copy to avoid modifying the original DataFrame in place
    df.loc[df['Tier'] == 'Action@MOT', 'Annotation'] = df[df['Tier'] == 'Action@MOT']['Annotation'].str.replace('ball', 'main_object', regex=False)
    df.loc[df['Tier'] == 'Action@MOT', 'Annotation'] = df[df['Tier'] == 'Action@MOT']['Annotation'].str.replace('toy pyramid', 'object', regex=False)
    df.loc[df['Tier'] == 'Action@MOT', 'Annotation'] = df[df['Tier'] == 'Action@MOT']['Annotation'].str.replace('glass', 'object', regex=False)
    df.loc[df['Tier'] == 'Action@MOT', 'Annotation'] = df[df['Tier'] == 'Action@MOT']['Annotation'].str.replace('toy', 'object', regex=False)
    return df

In [20]:
df = standardize_action_annotations(df)

### Applying categorization on df

In [21]:
df['Annotation'] = df.apply(
    lambda row: categorize_facial_mot(row['Annotation']) if row['Tier'] == 'Facial@MOT' else
                categorize_utterance_mot(row['Annotation']) if row['Tier'] == 'Utterance@MOT' else
                categorize_gaze_mot(row['Annotation']) if row['Tier'] == 'Gaze@MOT' else
                categorize_gaze_chi(row['Annotation']) if row['Tier'] == 'Gaze@CHI' else
                row['Annotation'],
    axis=1
)

## Create objects

### Standard

In [22]:
tiers = [
    "Action@MOT",
    "Gaze@MOT",
    "Utterance@MOT",
    "Prosody@MOT",
    "Facial@MOT",
    "Laughter@MOT",
    "Laughter@CHI",
    "Gaze@CHI"
]

pivot_df = df.pivot_table(
    index='Time',
    columns='Tier',
    values='Annotation',
    aggfunc=lambda x: x.iloc[0]
)

pivot_df = pivot_df[tiers]

pivot_df = pivot_df.where(pd.notnull(pivot_df), None)

records = pivot_df.to_dict(orient='records')

X = records.copy()

### With GazeRelation

In [23]:
tiersGR = [
    "Action@MOT",
    "Gaze@MOT",
    "Utterance@MOT",
    "Prosody@MOT",
    "Facial@MOT",
    "Laughter@MOT",
    "Laughter@CHI",
    "Gaze@CHI",
    "GazeRelation"
]
pivot_gr_df = df.pivot_table(
    index='Time',
    columns='Tier',
    values='Annotation',
    aggfunc=lambda x: x.iloc[0]
)

records_gr = pivot_gr_df.to_dict(orient='records')

for record in records_gr:
    mot_target = record.get("Gaze@MOT")
    chi_target = record.get("Gaze@CHI")

    if chi_target == "mom" and mot_target == "child":
        label = "MA"  # MutualAttention
    elif mot_target == "invisible":
        label = "<UNK>"  # Unknown
    elif chi_target == "mom":
        label = "SA(CHI)"  # SingleAttention(CHI)
    elif mot_target == "child":
        label = "SA(MOT)"  # SingleAttention(MOT)
    elif chi_target == mot_target and chi_target is not None:
        label = "ShA"  # SharedAttention
    elif chi_target != mot_target:
        label = "L"  # LostAttention
    else:
        label = "<UNK>"  # Unknown

    record["GazeRelation"] = label

pivot_gr_df = pd.DataFrame(records_gr)

pivot_gr_df = pivot_gr_df[tiersGR]

pivot_gr_df = pivot_gr_df.where(pd.notnull(pivot_gr_df), None)

records_gr = pivot_gr_df.to_dict(orient='records')

X_gr = records_gr.copy()

## Retrieve vocabs

In [24]:
values = defaultdict(set)

for r in records_gr:
    for key in r.keys():
        value = r[key]
        if value is None or value == '' or value == 'invisible':
            value = "<UNK>"  # use <UNK> token for missing
        values[key].add(value)

vocab_action    = {token: idx for idx, token in enumerate(sorted(values['Action@MOT']))}
vocab_gaze      = {token: idx for idx, token in enumerate(sorted(values['Gaze@MOT']))}
vocab_utterance = {token: idx for idx, token in enumerate(sorted(values['Utterance@MOT']))}
vocab_prosody   = {token: idx for idx, token in enumerate(sorted(values['Prosody@MOT']))}
vocab_facial    = {token: idx for idx, token in enumerate(sorted(values['Facial@MOT']))}
vocab_laughter  = {token: idx for idx, token in enumerate(sorted(values['Laughter@MOT']))}
vocab_laughter_chi = {token: idx for idx, token in enumerate(sorted(values['Laughter@CHI']))}
vocab_gaze_chi  = {token: idx for idx, token in enumerate(sorted(values['Gaze@CHI']))}
vocab_gazerelation  = {token: idx for idx, token in enumerate(sorted(values['GazeRelation']))}

In [25]:
vocabs = {"action": vocab_action, "gaze": vocab_gaze, "utterance": vocab_utterance,
          "prosody": vocab_prosody, "facial": vocab_facial, "laughter": vocab_laughter,
          "laughter_chi": vocab_laughter_chi, "gaze_chi": vocab_gaze_chi}

vocabs_gr = {"action": vocab_action, "gaze": vocab_gaze, "utterance": vocab_utterance,
             "prosody": vocab_prosody, "facial": vocab_facial, "laughter": vocab_laughter,
             "gazerelation": vocab_gazerelation, "laughter_chi": vocab_laughter_chi, "gaze_chi": vocab_gaze_chi}

# Helper functions

In [26]:
def get_data(isGazeRelation):
    if isGazeRelation:
        return X_gr, vocabs_gr
    else:
        return X, vocabs

## Data Loader (batches divsion)

In [27]:
from torch.utils.data import DataLoader

# Finding the index of a value in a vocabulary
def lookup(vocab, value):
    if value is None:
        value = "<UNK>"
    return vocab.get(value, vocab["<UNK>"])

def collate_fn(batch):
    """
    Collate function for DataLoader without GazeRelation target.
    Processes a batch of data into input sequences and target tensors.
    """
    sequences, targets_list = zip(*batch)

    target_laughter_indices = []
    target_gaze_indices = []

    # Convert each target value to an index
    for target_dict in targets_list:
        laughter_val = target_dict.get('Laughter@CHI')
        gaze_val = target_dict.get('Gaze@CHI')

        target_laughter_indices.append(lookup(vocab_laughter_chi, laughter_val))
        target_gaze_indices.append(lookup(vocab_gaze_chi, gaze_val))

    # Convert target index lists to tensors
    targets_laughter = torch.tensor(target_laughter_indices, dtype=torch.long)
    targets_gaze = torch.tensor(target_gaze_indices, dtype=torch.long)

    # Wrap each sequence dict in a list to create a sequence of length 1
    X_batch_sequences = [[seq_dict] for seq_dict in sequences]

    return X_batch_sequences, (targets_gaze, targets_laughter)

def collate_fn_gr(batch):
    """
    Collate function for DataLoader with GazeRelation target.
    Same as collate_fn but also processes the 'GazeRelation' target.
    """
    sequences, targets_list = zip(*batch)

    target_laughter_indices = []
    target_gaze_indices = []
    target_gazerelation_indices = []

    for target_dict in targets_list:
        laughter_val = target_dict.get('Laughter@CHI')
        gaze_val = target_dict.get('Gaze@CHI')
        gazerelation_val = target_dict.get('GazeRelation')

        target_laughter_indices.append(lookup(vocab_laughter_chi, laughter_val))
        target_gaze_indices.append(lookup(vocab_gaze_chi, gaze_val))
        target_gazerelation_indices.append(lookup(vocab_gazerelation, gazerelation_val))

    # Convert lists to tensors
    targets_laughter = torch.tensor(target_laughter_indices, dtype=torch.long)
    targets_gaze = torch.tensor(target_gaze_indices, dtype=torch.long)
    targets_gazerelation = torch.tensor(target_gazerelation_indices, dtype=torch.long)

    # Wrap each sequence dict in a list to create a sequence of length 1
    X_batch_sequences = [[seq_dict] for seq_dict in sequences]

    return X_batch_sequences, (targets_gaze, targets_laughter, targets_gazerelation)

def get_loader(X, y, batch_size, isGazeRelation):
    """
    Creates a DataLoader for the given dataset.
    """
    dataset = MultimodalDataset(X, y)
    if isGazeRelation:
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn_gr)
    else:
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    return loader


In [28]:
def get_loaders(X, isGazeRelation, isMixedPhases, train_size, val_size, n_parts):
    if not isMixedPhases:
        n_parts = 1

    X_train, y_train, X_val, y_val, X_test, y_test = DatasetSplit(dataset = X, n_parts = n_parts, train_size = train_size, val_size = val_size).main()

    train_batch_size = len(X_train) // n_parts
    val_batch_size = len(X_val) // n_parts
    test_batch_size = len(X_test) // n_parts

    train_loader = get_loader(X_train, y_train, batch_size=train_batch_size, isGazeRelation=isGazeRelation)
    val_loader = get_loader(X_val, y_val, batch_size=val_batch_size, isGazeRelation=isGazeRelation)
    test_loader = get_loader(X_test, y_test, batch_size=test_batch_size, isGazeRelation=isGazeRelation)

    return train_loader, val_loader, test_loader

## Evaluation Loop

In [29]:
def evaluation_loop(model, loader, criterion, criterion_gaze, isGazeRelation):
    if isGazeRelation:
        return evaluation_on_set_gr(model, loader, criterion, criterion_gaze)
    else:
        return evaluation_on_set(model, loader, criterion, criterion_gaze)

In [30]:
def get_closest_gaze_annotation(predicted_embedding, gaze_embeddings_tensor, gaze_annotations):
    """Finds the closest gaze annotation for a given predicted embedding."""
    predicted_embedding_normalized = F.normalize(predicted_embedding.unsqueeze(0), dim=1)
    gaze_embeddings_tensor_normalized = F.normalize(gaze_embeddings_tensor, dim=1)

    # Calculate cosine similarity between the predicted embedding and all gaze embeddings
    similarity_scores = cosine_similarity(predicted_embedding_normalized, gaze_embeddings_tensor_normalized)

    # Convert similarity_scores to a PyTorch tensor
    similarity_scores_tensor = torch.tensor(similarity_scores)

    # Get the index of the highest similarity score
    closest_index = torch.argmax(similarity_scores_tensor).item()

    # Return the corresponding annotation
    return gaze_annotations[closest_index]

In [31]:
def evaluation_on_set(model, loader, criterion, criterion_gaze):
        total_loss = 0
        targets_gazes = []
        preds_gazes = []
        targets_laughters = []
        preds_laughters = []

        for X_batch, (targets_gaze, targets_laughter) in loader:
            predicted_gaze, logits_laughter = model(X_batch, device)

            # Create a target tensor of ones for cosine similarity loss
            similarity_target = torch.ones(predicted_gaze.size(0), device=device)

            # Convert target gaze indices to their annotation strings
            gaze_chi_annotations = list(vocab_gaze_chi.keys())
            target_gaze_annotations = [gaze_chi_annotations[idx] for idx in targets_gaze.tolist()]

            # Encode target gaze annotations into embeddings
            target_gaze_embeddings = torch.tensor(
                model.sentence_encoder.encode(target_gaze_annotations),
                dtype=torch.float32
            ).to(device)

            # Calculate gaze similarity loss
            loss_gaze = criterion_gaze(predicted_gaze, target_gaze_embeddings, similarity_target)

            # Calculate laughter classification loss
            loss_laughter = criterion(logits_laughter, targets_laughter.to(device))

            total_loss += (loss_gaze + loss_laughter).item()

            # Precompute all possible gaze embeddings for prediction matching
            gaze_chi_annotations = list(vocab_gaze_chi.keys())
            gaze_chi_embeddings = model.sentence_encoder.encode(gaze_chi_annotations)
            gaze_chi_embeddings_tensor = torch.tensor(gaze_chi_embeddings).to(device)

            # Map each predicted embedding to the closest annotation string
            predicted_gaze_annotations = [
                get_closest_gaze_annotation(pred_embed, gaze_chi_embeddings_tensor, gaze_chi_annotations)
                for pred_embed in predicted_gaze
            ]

            # Convert predicted annotations back to vocabulary indices
            predicted_gaze_indices = [
                vocab_gaze_chi.get(anno, vocab_gaze_chi["<UNK>"])
                for anno in predicted_gaze_annotations
            ]

            targets_gazes.extend(targets_gaze.cpu().numpy())
            preds_gazes.extend(predicted_gaze_indices)

            targets_laughters.extend(targets_laughter.cpu().numpy())
            preds_laughters.extend(torch.argmax(logits_laughter, dim=1).cpu().numpy())

        avg_loss = total_loss / len(loader)

        gaze_accuracy, gaze_precision, gaze_f1 = get_quality_metrics(preds_gazes, targets_gazes)

        laughter_accuracy, laughter_precision, laughter_f1 = get_quality_metrics(preds_laughters, targets_laughters)


        return avg_loss, loss_gaze, loss_laughter, gaze_accuracy, gaze_precision, gaze_f1, laughter_accuracy, laughter_precision, laughter_f1

In [32]:
def evaluation_on_set_gr(model, loader, criterion, criterion_gaze):
        total_loss = 0
        targets_gazes = []
        preds_gazes = []
        targets_laughters = []
        preds_laughters = []
        targets_gazerelations = []
        preds_gazerelations = []

        for X_batch, (targets_gaze, targets_laughter, targets_gazerelation) in loader:
            predicted_gaze, logits_laughter, logits_gazerelation = model(X_batch, device)

            # Create "ideal" similarity targets (all ones) for gaze embedding comparison
            similarity_target = torch.ones(predicted_gaze.size(0), device=device)

            # Map gaze target indices to their annotation strings
            gaze_chi_annotations = list(vocab_gaze_chi.keys())
            target_gaze_annotations = [gaze_chi_annotations[idx] for idx in targets_gaze.tolist()]

            # Encode target gaze annotations into embeddings
            target_gaze_embeddings = torch.tensor(
                model.sentence_encoder.encode(target_gaze_annotations),
                dtype=torch.float32
            ).to(device)

            loss_gaze = criterion_gaze(predicted_gaze, target_gaze_embeddings, similarity_target)
            loss_laughter = criterion(logits_laughter, targets_laughter.to(device))
            loss_gazerelation = criterion(logits_gazerelation, targets_gazerelation.to(device))

            total_loss += (loss_gaze + loss_laughter + loss_gazerelation).item()

            # Precompute all possible gaze embeddings for matching predictions
            gaze_chi_embeddings = model.sentence_encoder.encode(gaze_chi_annotations)
            gaze_chi_embeddings_tensor = torch.tensor(gaze_chi_embeddings).to(device)

            # Map each predicted gaze embedding to the closest annotation string
            predicted_gaze_annotations = [
                get_closest_gaze_annotation(pred_embed, gaze_chi_embeddings_tensor, gaze_chi_annotations)
                for pred_embed in predicted_gaze
            ]

            # Convert predicted annotations back to vocabulary indices for metrics
            predicted_gaze_indices = [
                vocab_gaze_chi.get(anno, vocab_gaze_chi["<UNK>"])
                for anno in predicted_gaze_annotations
            ]

            # Store targets and predictions for all tasks
            targets_gazes.extend(targets_gaze.cpu().numpy())
            preds_gazes.extend(predicted_gaze_indices)

            targets_laughters.extend(targets_laughter.cpu().numpy())
            preds_laughters.extend(torch.argmax(logits_laughter, dim=1).cpu().numpy())

            targets_gazerelations.extend(targets_gazerelation.cpu().numpy())
            preds_gazerelations.extend(torch.argmax(logits_gazerelation, dim=1).cpu().numpy())

        # Compute average loss across batches
        avg_loss = total_loss / len(loader)

        # Compute evaluation metrics for gaze and laughter
        gaze_accuracy, gaze_precision, gaze_f1 = get_quality_metrics(preds_gazes, targets_gazes)
        laughter_accuracy, laughter_precision, laughter_f1 = get_quality_metrics(preds_laughters, targets_laughters)

        # gazerelation_accuracy, gazerelation_precision, gazerelation_f1 = get_quality_metrics(preds_gazerelations, targets_gazerelations)

        return avg_loss, loss_gaze, loss_laughter, gaze_accuracy, gaze_precision, gaze_f1, laughter_accuracy, laughter_precision, laughter_f1

## Training Loop

In [33]:
def train_loop(model, loader, criterion, criterion_gaze, optimizer, isGazeRelation):
        if isGazeRelation:
            return train_loop_gr(model, loader, criterion, criterion_gaze, optimizer)
        else:
            return train_loop_standard(model, loader, criterion, criterion_gaze, optimizer)

In [34]:
def train_loop_gr(model, loader, criterion, criterion_gaze, optimizer):
    total_loss = 0
    for X_batch, (targets_gaze, targets_laughter, targets_gazerelation) in loader:

        targets_gaze = targets_gaze.to(device)
        targets_laughter = targets_laughter.to(device)
        targets_gazerelation = targets_gazerelation.to(device)

        predicted_gaze, logits_laughter, logits_gazerelation = model(X_batch, device)

        # The target for CosineEmbeddingLoss is 1 for similar embeddings
        similarity_target = torch.ones(predicted_gaze.size(0), device=device)

        gaze_chi_annotations = list(vocab_gaze_chi.keys())
        target_gaze_annotations = [gaze_chi_annotations[idx] for idx in targets_gaze.tolist()]
        target_gaze_embeddings = torch.tensor(model.sentence_encoder.encode(target_gaze_annotations), dtype=torch.float32).to(device)

        loss_gaze = criterion_gaze(predicted_gaze, target_gaze_embeddings, similarity_target)
        loss_laughter = criterion(logits_laughter, targets_laughter)
        loss_gazerelation = criterion(logits_gazerelation, targets_gazerelation)

        loss = loss_gaze + loss_laughter + loss_gazerelation

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return model

In [35]:
def train_loop_standard(model, loader, criterion, criterion_gaze, optimizer):
        for X_batch, (targets_gaze, targets_laughter) in loader:
            targets_gaze = targets_gaze.to(device)
            targets_laughter = targets_laughter.to(device)

            predicted_gaze, logits_laughter = model(X_batch, device)

            similarity_target = torch.ones(predicted_gaze.size(0), device=device)

            gaze_chi_annotations = list(vocab_gaze_chi.keys())
            target_gaze_annotations = [gaze_chi_annotations[idx] for idx in targets_gaze.tolist()]
            target_gaze_embeddings = torch.tensor(model.sentence_encoder.encode(target_gaze_annotations), dtype=torch.float32).to(device)

            loss_gaze = criterion_gaze(predicted_gaze, target_gaze_embeddings, similarity_target)

            loss_laughter = criterion(logits_laughter, targets_laughter.to(device))

            loss = loss_gaze + loss_laughter

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        return model

## Training Model

In [36]:
@dataclass
class DatasetParams:
    isGazeRelation: bool
    isMixedPhases: bool
    train_size: int
    val_size: int
    n_parts: int

@dataclass
class ModelParams:
    hidden_dim: int
    num_layers: int
    dropout: float
    input_size: int
    bidirectional: bool

@dataclass
class TrainParams:
    lr: float
    weight_decay: float
    num_epochs: int

In [37]:

def training_model(model, dataset_params: DatasetParams, model_params: ModelParams, train_params: TrainParams):
    dataset, vocabs = get_data(dataset_params.isGazeRelation)

    model = model(
        vocabs,
        dataset_params.isGazeRelation,
        hidden_dim = model_params.hidden_dim,
        num_layers =  model_params.num_layers,
        dropout = model_params.dropout if model_params.num_layers > 1 else 0.0,
        bidirectional = model_params.bidirectional,
        input_size = model_params.input_size
    ).to(device)

    train_loader, val_loader, test_loader = get_loaders(dataset, dataset_params.isGazeRelation, dataset_params.isMixedPhases, dataset_params.train_size, dataset_params.val_size, dataset_params.n_parts)

    optimizer = optim.Adam(model.parameters(), lr = train_params.lr, weight_decay = train_params.weight_decay)
    criterion = nn.CrossEntropyLoss()
    criterion_emb = nn.CosineEmbeddingLoss()
    cosine_similarity = nn.CosineSimilarity(dim=1)

    num_epochs = train_params.num_epochs

    print("Starting training...")
    for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
        model.train()

        model = train_loop(model, train_loader, criterion, criterion_emb, optimizer, dataset_params.isGazeRelation)

        model.eval()
        with torch.no_grad():
            avg_train_loss, loss_gaze, loss_laughter, train_gaze_accuracy, train_gaze_precision, train_gaze_f1, train_laughter_accuracy, train_laughter_precision, train_laughter_f1 = evaluation_loop(model, train_loader, criterion, criterion_emb, dataset_params.isGazeRelation)

        # Validation
        model.eval()
        with torch.no_grad():
            avg_val_loss, loss_gaze, loss_laughter, val_gaze_accuracy, val_gaze_precision, val_gaze_f1, val_laughter_accuracy, val_laughter_precision, val_laughter_f1 = evaluation_loop(model, val_loader, criterion, criterion_emb, dataset_params.isGazeRelation)

        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"\tTrain Loss: {avg_train_loss:.4f},  Train Gaze Acc: {train_gaze_accuracy:.4f}, Train Gaze Prec: {train_gaze_precision:.4f}, Train Gaze F1: {train_gaze_f1:.4f}, Train Laughter Acc: {train_laughter_accuracy:.4f}, Train Laughter Prec: {train_laughter_precision:.4f}, Train Laughter F1: {train_laughter_f1:.4f}, "
              f"\n\tVal Loss: {avg_val_loss:.4f}, Val Gaze Acc: {val_gaze_accuracy:.4f}, Val Gaze Prec: {val_gaze_precision:.4f}, Val Gaze F1: {val_gaze_f1:.4f}, Val Laughter Acc: {val_laughter_accuracy:.4f}, Val Laughter Prec: {val_laughter_precision:.4f}, Val Laughter F1: {val_laughter_f1:.4f}"
              )

    print("Training finished.")

    # Evaluate on Test Set
    model.eval()
    with torch.no_grad():
        avg_test_loss, loss_gaze, loss_laughter, test_accuracy_gaze, test_gaze_precision, test_gaze_f1, test_accuracy_laughter, test_laughter_precision, test_laughter_f1 = evaluation_loop(model, test_loader, criterion, criterion_emb, dataset_params.isGazeRelation)


    print("\nTest Set Evaluation:")
    print(f"Test Loss: {avg_test_loss:.4f}, Test Gaze Acc: {test_accuracy_gaze:.4f}, Test Gaze Prec: {test_gaze_precision:.4f}, Test Gaze F1: {test_gaze_f1:.4f}, Test Laughter Acc: {test_accuracy_laughter:.4f}, Test Laughter Prec: {test_laughter_precision:.4f}, Test Laughter F1: {test_laughter_f1:.4f}")

# Single LSTM (with Gaze Embedding)

## Model

In [38]:
class EnfantReactionPredictor(nn.Module):
    """
      Multi-LSTM model to predict:
      - Child gaze embedding (semantic - SentenceTransformer)
      - Child laughter category
      - Optionally, gaze relation category
    """
    def __init__(self,
                 vocabs,
                 isGazeRelation,
                 hidden_dim = 16,
                 num_layers = 2,
                 dropout = 0.0,
                 input_size = 40,
                 bidirectional = False
                 ):

        super().__init__()

        self.isGazeRelation = isGazeRelation

        # --- Store vocab dictionaries for categorical features ---
        self.vocab_gaze = vocabs["gaze"]
        self.vocab_utterance = vocabs["utterance"]
        self.vocab_prosody = vocabs["prosody"]
        self.vocab_facial = vocabs["facial"]
        self.vocab_laughter = vocabs["laughter"]
        self.vocab_laughter_chi = vocabs["laughter_chi"]

        if self.isGazeRelation:
            self.vocab_gazerelation = vocabs["gazerelation"]

        # --- Embedding layers for each categorical feature ---
        self.embed_gaze = nn.Embedding(len(vocab_gaze), 8)
        self.embed_utterance = nn.Embedding(len(vocab_utterance), 8)
        self.embed_prosody = nn.Embedding(len(vocab_prosody), 4)
        self.embed_facial = nn.Embedding(len(vocab_facial), 8)
        self.embed_laughter = nn.Embedding(len(vocab_laughter), 4)

        # Pre-trained semantic encoder for "Action@MOT" (mother's action description)
        self.sentence_encoder = SentenceTransformer('all-MiniLM-L6-v2')

        self.embed_laughter_chi = nn.Embedding(len(vocab_laughter_chi), 4)

        if self.isGazeRelation:
            self.embed_gazerelation = nn.Embedding(len(self.vocab_gazerelation), 4)

        # --- LSTM sequence model ---
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=bidirectional
        )

        self.dropout = nn.Dropout(dropout)

        # Determine decoder input dimension based on LSTM directionality
        decoder_input_dim = hidden_dim * (2 if bidirectional else 1)

        # ---  decoders ---
        self.decoder_laughter = nn.Linear(decoder_input_dim, len(vocab_laughter_chi))

        # Project LSTM output into SentenceTransformer embedding space for gaze prediction
        self.project_gaze = nn.Linear(
            decoder_input_dim,
            self.sentence_encoder.get_sentence_embedding_dimension()
        )

        if self.isGazeRelation:
            self.decoder_gazerelation = nn.Linear(decoder_input_dim, len(self.vocab_gazerelation))

    def lookup(self, vocab, value):

        if value is None:
            value = "<UNK>"
        return vocab.get(value, vocab["<UNK>"])

    def encode_input_batch(self, batch_sequence_of_dicts, device):

        all_seqs = []

        for sequence in batch_sequence_of_dicts:
            seq = []

            for step_dict in sequence:
                # Categorical embeddings
                gaze = torch.tensor([self.lookup(self.vocab_gaze, step_dict.get("Gaze@MOT"))], dtype=torch.long)
                utterance = torch.tensor([self.lookup(self.vocab_utterance, step_dict.get("Utterance@MOT"))], dtype=torch.long)
                prosody = torch.tensor([self.lookup(self.vocab_prosody, step_dict.get("Prosody@MOT"))], dtype=torch.long)
                facial = torch.tensor([self.lookup(self.vocab_facial, step_dict.get("Facial@MOT"))], dtype=torch.long)
                laughter = torch.tensor([self.lookup(self.vocab_laughter, step_dict.get("Laughter@MOT"))], dtype=torch.long)

                # Sentence embedding for "Action@MOT" (not trainable here)
                action_text = step_dict.get("Action@MOT") or "<UNK>"
                encoded_action = torch.tensor(
                    self.sentence_encoder.encode(action_text),
                    dtype=torch.float32
                ).unsqueeze(0)

                # Concatenate all features into one vector
                step_emb = torch.cat([
                    self.embed_gaze(gaze),
                    self.embed_utterance(utterance),
                    self.embed_prosody(prosody),
                    self.embed_facial(facial),
                    self.embed_laughter(laughter),
                    encoded_action
                ], dim=-1)

                seq.append(step_emb)

            # Concatenate over time dimension
            seq = torch.cat(seq, dim=0)
            all_seqs.append(seq)

        # Stack into batch tensor
        batch_tensor = torch.stack(all_seqs).to(device)
        return batch_tensor

    def forward(self, batch_sequence_of_dicts, device):

        encoded_steps = self.encode_input_batch(batch_sequence_of_dicts, device)

        output, (h_n, c_n) = self.lstm(encoded_steps)

        output_last = output.squeeze(1)

        logits_laughter = self.decoder_laughter(output_last)
        predicted_gaze_embeddings = self.project_gaze(output_last)

        if self.isGazeRelation:
            logits_gazerelation = self.decoder_gazerelation(output_last)
            return predicted_gaze_embeddings, logits_laughter, logits_gazerelation

        return predicted_gaze_embeddings, logits_laughter


In [39]:
training_model(EnfantReactionPredictor,
               DatasetParams(isGazeRelation=True, isMixedPhases=True, train_size=0.7, val_size=0.15, n_parts=4),
               ModelParams(hidden_dim = 256, num_layers = 1, dropout = 0.2, bidirectional=False, input_size=416),
               TrainParams(lr = 0.001, weight_decay = 1e-6, num_epochs = 1)
               )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Starting training...


Training Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch [1/1], 	Train Loss: 2.7743,  Train Gaze Acc: 0.5138, Train Gaze Prec: 0.2643, Train Gaze F1: 0.3490, Train Laughter Acc: 0.7399, Train Laughter Prec: 0.5474, Train Laughter F1: 0.6293, 
	Val Loss: 2.8547, Val Gaze Acc: 0.5222, Val Gaze Prec: 0.2870, Val Gaze F1: 0.3704, Val Laughter Acc: 0.5323, Val Laughter Prec: 0.2833, Val Laughter F1: 0.3698
Training finished.

Test Set Evaluation:
Test Loss: 2.9268, Test Gaze Acc: 0.4960, Test Gaze Prec: 0.2460, Test Gaze F1: 0.3289, Test Laughter Acc: 0.3631, Test Laughter Prec: 0.1348, Test Laughter F1: 0.1966


## Grid Search

In [40]:
from sklearn.model_selection import ParameterGrid

def grid_search(model, dataset_params: DatasetParams, param_grid: dict):

    best_val_loss = float('inf')
    best_params = None
    results = []

    dataset, vocabs = get_data(dataset_params.isGazeRelation)

    param_grid = ParameterGrid(param_grid)
    print(f"Starting Grid Search with {len(param_grid)} combinations...")

    for i, params in enumerate(tqdm(param_grid, desc="Processing params", unit="set")):
        print(f"\n--- Running combination {i+1}/{len(param_grid)} ---")
        print(f"Parameters: {params}")

        model_params = ModelParams(
            hidden_dim=params['hidden_dim'],
            num_layers=params['num_layers'],
            dropout=params['dropout'],
            bidirectional=params['bidirectional'],
            input_size=416
        )
        train_params = TrainParams(
            lr=params['lr'],
            weight_decay=params['weight_decay'],
            num_epochs=params['num_epochs']
        )

        model_instance = model(
            vocabs,
            dataset_params.isGazeRelation,
            hidden_dim=model_params.hidden_dim,
            num_layers=model_params.num_layers,
            dropout=model_params.dropout,
            bidirectional=model_params.bidirectional,
            input_size=model_params.input_size
        ).to(device)

        train_loader, val_loader, test_loader = get_loaders(
            dataset,
            dataset_params.isGazeRelation,
            dataset_params.isMixedPhases,
            dataset_params.train_size,
            dataset_params.val_size,
            dataset_params.n_parts
        )

        optimizer = optim.Adam(model_instance.parameters(), lr=train_params.lr, weight_decay=train_params.weight_decay)
        criterion = nn.CrossEntropyLoss()       # for laughter classification
        criterion_gaze = nn.CosineEmbeddingLoss()  # for gaze embedding similarity

        # ------------------ Training Loop ------------------
        for epoch in tqdm(range(train_params.num_epochs), desc=f"Training Epochs (Params {params})"):
            # Training phase
            model_instance.train()
            model_instance = train_loop(
                model_instance,
                train_loader,
                criterion,
                criterion_gaze,
                optimizer,
                dataset_params.isGazeRelation
            )

            # Validation phase
            model_instance.eval()
            with torch.no_grad():
                avg_val_loss, _, _, val_gaze_accuracy, val_gaze_precision, val_gaze_f1, \
                val_laughter_accuracy, val_laughter_precision, val_laughter_f1 = evaluation_loop(
                    model_instance,
                    val_loader,
                    criterion,
                    criterion_gaze,
                    dataset_params.isGazeRelation
                )

        # ------------------ Final Evaluation on Test Set ------------------
        model_instance.eval()
        with torch.no_grad():
            avg_val_loss, _, _, val_gaze_accuracy, val_gaze_precision, val_gaze_f1, \
            val_laughter_accuracy, val_laughter_precision, val_laughter_f1 = evaluation_loop(
                model_instance,
                test_loader,
                criterion,
                criterion_gaze,
                dataset_params.isGazeRelation
            )

        print(f"Validation Loss for this combo: {avg_val_loss:.4f}")

        results.append({
            'params': params,
            'val_loss': avg_val_loss,
            'val_gaze_accuracy': val_gaze_accuracy,
            'val_gaze_precision': val_gaze_precision,
            'val_gaze_f1': val_gaze_f1,
            'val_laughter_accuracy': val_laughter_accuracy,
            'val_laughter_precision': val_laughter_precision,
            'val_laughter_f1': val_laughter_f1,
        })

        # Update best parameters if this run performed better
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_params = params
            # Optional: save best model weights
            # torch.save(model_instance.state_dict(), 'best_model_state_dict.pth')

    # ------------------ Summary ------------------
    print("\n--- Grid Search Finished ---")
    print(f"Best parameters found: {best_params}")
    print(f"Best validation loss: {best_val_loss:.4f}")

    return results, best_params


### Results - Standard

In [41]:
# Define the parameter grid
param_grid = {
    'hidden_dim': [128, 256],
    'num_layers': [1, 2],
    'dropout': [0.1, 0.3],
    'bidirectional': [False, True],
    'lr': [0.003],
    'weight_decay': [1e-5],
    'num_epochs': [10]
}

In [42]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

# Run the grid search
results, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results:
      print(res)

results_df = pd.DataFrame(results)

print("\nGrid Search Results DataFrame:")
display(results_df)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8823

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.7738

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 0.7738

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.8643853068351746, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_gaze_f1': 0.07310604613856848, 'val_laughter_accuracy': np.float64(0.8629807692307693), 'val_laughter_precision': 0.8932520211262462, 'val_laughter_f1': 0.8752619715222879}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.8866450786590576, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.864385,0.210337,0.044241,0.073106,0.862981,0.893252,0.875262
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.886645,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.849088,0.210337,0.044241,0.073106,0.721154,0.866108,0.773109
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.982254,0.210337,0.044241,0.073106,0.733173,0.849127,0.778793
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.844300,0.210337,0.044241,0.073106,0.796875,0.860194,0.823129
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.889345,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.862108,0.210337,0.044241,0.073106,0.731971,0.848937,0.777946
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.906027,0.210337,0.044241,0.073106,0.706731,0.872242,0.763285
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.854109,0.210337,0.044241,0.073106,0.751202,0.870522,0.794926
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.890885,0.210337,0.044241,0.073106,0.727163,0.866969,0.777501


In [ ]:
results_df.to_csv('grid_search_results_gaze_emb_results_df.csv', index=False)

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

# Run the grid search
results2, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results2:
      print(res)

results_df_2 = pd.DataFrame(results2)

print("\nGrid Search Results DataFrame:")
display(results_df_2)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.1494

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.1692

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 1.1494

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.3589394092559814, 'val_gaze_accuracy': np.float64(0.49603174603174605), 'val_gaze_precision': 0.24604749307130258, 'val_gaze_f1': 0.328933518588691, 'val_laughter_accuracy': np.float64(0.3630952380952381), 'val_laughter_precision': 0.13480182492158538, 'val_laughter_f1': 0.19661052193803286}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.4693374633789062, 'val_gaze_accuracy': np.float64(0.49603174603174605), 'val_gaze_precision': 0.24604749307130258, 'val_

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.358939,0.496032,0.246047,0.328934,0.363095,0.134802,0.196611
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.469337,0.496032,0.246047,0.328934,0.369048,0.136196,0.198965
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.312282,0.496032,0.246047,0.328934,0.369048,0.136196,0.198965
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.236566,0.496032,0.246047,0.328934,0.478175,0.698230,0.419939
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.382076,0.496032,0.246047,0.328934,0.369048,0.136196,0.198965
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.538490,0.496032,0.246047,0.328934,0.369048,0.136196,0.198965
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.389385,0.480159,0.294938,0.365418,0.490079,0.586616,0.479897
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.312604,0.496032,0.246047,0.328934,0.442460,0.457813,0.448965
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.360281,0.496032,0.246047,0.328934,0.432540,0.626695,0.355077
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.212256,0.496032,0.246047,0.328934,0.462302,0.482432,0.470016


In [ ]:
results_df.to_csv('grid_search_results_gaze_emb_results_df_2.csv', index=False)

                                               params  val_loss  \
0   {'bidirectional': False, 'dropout': 0.1, 'hidd...  0.742618   
1   {'bidirectional': False, 'dropout': 0.1, 'hidd...  0.725145   
2   {'bidirectional': False, 'dropout': 0.1, 'hidd...  0.706039   
3   {'bidirectional': False, 'dropout': 0.1, 'hidd...  0.721333   
4   {'bidirectional': False, 'dropout': 0.3, 'hidd...  0.728857   
5   {'bidirectional': False, 'dropout': 0.3, 'hidd...  0.745268   
6   {'bidirectional': False, 'dropout': 0.3, 'hidd...  0.740294   
7   {'bidirectional': False, 'dropout': 0.3, 'hidd...  0.744448   
8   {'bidirectional': True, 'dropout': 0.1, 'hidde...  0.712053   
9   {'bidirectional': True, 'dropout': 0.1, 'hidde...  0.717700   
10  {'bidirectional': True, 'dropout': 0.1, 'hidde...  0.743451   
11  {'bidirectional': True, 'dropout': 0.1, 'hidde...  0.716229   
12  {'bidirectional': True, 'dropout': 0.3, 'hidde...  0.732423   
13  {'bidirectional': True, 'dropout': 0.3, 'hidde...  0.72305

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.6,
    val_size=0.2,
    n_parts=8
)

# Run the grid search
results3, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results3:
      print(res)

results_df_3 = pd.DataFrame(results3)

print("\nGrid Search Results DataFrame:")
display(results_df_3)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.9781

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.0017

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 0.9655

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.9873763653967116, 'val_gaze_accuracy': np.float64(0.7157894736842105), 'val_gaze_precision': 0.7579177535675925, 'val_gaze_f1': 0.7041320309571583, 'val_laughter_accuracy': np.float64(0.7142857142857143), 'val_laughter_precision': 0.7417079540957657, 'val_laughter_f1': 0.7006164438175525}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.9949998325771756, 'val_gaze_accuracy': np.float64(0.7218045112781954), 'val_gaze_precision': 0.7696360944792294, 'val_gaze_

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.987376,0.715789,0.757918,0.704132,0.714286,0.741708,0.700616
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.995000,0.721805,0.769636,0.709503,0.711278,0.723982,0.702957
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.989088,0.721805,0.769636,0.709503,0.694737,0.712133,0.682846
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.006840,0.721805,0.769636,0.709503,0.706767,0.720088,0.697826
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.992986,0.715789,0.757918,0.704132,0.687218,0.701505,0.676020
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.021176,0.721805,0.769636,0.709503,0.694737,0.702425,0.687678
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.974932,0.718797,0.778112,0.707660,0.694737,0.712133,0.682846
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.980388,0.721805,0.769636,0.709503,0.709774,0.723435,0.700925
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.990611,0.715789,0.757918,0.704132,0.709774,0.724210,0.700594
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.000159,0.721805,0.769636,0.709503,0.711278,0.723982,0.702957


In [ ]:
import numpy as np
import pandas as pd

data = [
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.9873763653967116, 'val_gaze_accuracy': np.float64(0.7157894736842105), 'val_gaze_precision': 0.7579177535675925, 'val_gaze_f1': 0.7041320309571583, 'val_laughter_accuracy': np.float64(0.7142857142857143), 'val_laughter_precision': 0.7417079540957657, 'val_laughter_f1': 0.7006164438175525},
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.9949998325771756, 'val_gaze_accuracy': np.float64(0.7218045112781954), 'val_gaze_precision': 0.7696360944792294, 'val_gaze_f1': 0.709503414209287, 'val_laughter_accuracy': np.float64(0.7112781954887218), 'val_laughter_precision': 0.7239822743784896, 'val_laughter_f1': 0.7029566397195998},
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.989087700843811, 'val_gaze_accuracy': np.float64(0.7218045112781954), 'val_gaze_precision': 0.7696360944792294, 'val_gaze_f1': 0.709503414209287, 'val_laughter_accuracy': np.float64(0.6947368421052632), 'val_laughter_precision': 0.7121334460198863, 'val_laughter_f1': 0.6828455975246951},
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.0068404144710965, 'val_gaze_accuracy': np.float64(0.7218045112781954), 'val_gaze_precision': 0.7696360944792294, 'val_gaze_f1': 0.709503414209287, 'val_laughter_accuracy': np.float64(0.706766917293233), 'val_laughter_precision': 0.7200877698277953, 'val_laughter_f1': 0.6978258926558554},
{'params': {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.99298624197642, 'val_gaze_accuracy': np.float64(0.7157894736842105), 'val_gaze_precision': 0.7579177535675925, 'val_gaze_f1': 0.7041320309571583, 'val_laughter_accuracy': np.float64(0.687218045112782), 'val_laughter_precision': 0.7015054046494793, 'val_laughter_f1': 0.6760196788266963},
{'params': {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.0211764110459223, 'val_gaze_accuracy': np.float64(0.7218045112781954), 'val_gaze_precision': 0.7696360944792294, 'val_gaze_f1': 0.709503414209287, 'val_laughter_accuracy': np.float64(0.6947368421052632), 'val_laughter_precision': 0.7024251033952708, 'val_laughter_f1': 0.687677817751607},
{'params': {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.9749324454201592, 'val_gaze_accuracy': np.float64(0.718796992481203), 'val_gaze_precision': 0.7781120165306437, 'val_gaze_f1': 0.7076602397081814, 'val_laughter_accuracy': np.float64(0.6947368421052632), 'val_laughter_precision': 0.7121334460198863, 'val_laughter_f1': 0.6828455975246951},
{'params': {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.9803880055745443, 'val_gaze_accuracy': np.float64(0.7218045112781954), 'val_gaze_precision': 0.7696360944792294, 'val_gaze_f1': 0.709503414209287, 'val_laughter_accuracy': np.float64(0.7097744360902256), 'val_laughter_precision': 0.7234347389210353, 'val_laughter_f1': 0.7009251142696413},
{'params': {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.9906108445591397, 'val_gaze_accuracy': np.float64(0.7157894736842105), 'val_gaze_precision': 0.7579177535675925, 'val_gaze_f1': 0.7041320309571583, 'val_laughter_accuracy': np.float64(0.7097744360902256), 'val_laughter_precision': 0.7242101377474695, 'val_laughter_f1': 0.7005940631564308},
{'params': {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.0001590980423822, 'val_gaze_accuracy': np.float64(0.7218045112781954), 'val_gaze_precision': 0.7696360944792294, 'val_gaze_f1': 0.709503414209287, 'val_laughter_accuracy': np.float64(0.7112781954887218), 'val_laughter_precision': 0.7239822743784896, 'val_laughter_f1': 0.7029566397195998},
{'params': {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.9863053692711724, 'val_gaze_accuracy': np.float64(0.7218045112781954), 'val_gaze_precision': 0.7696360944792294, 'val_gaze_f1': 0.709503414209287, 'val_laughter_accuracy': np.float64(0.6962406015037594), 'val_laughter_precision': 0.7143054587048208, 'val_laughter_f1': 0.6842111429867872},
{'params': {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.9940315882364908, 'val_gaze_accuracy': np.float64(0.7218045112781954), 'val_gaze_precision': 0.7696360944792294, 'val_gaze_f1': 0.709503414209287, 'val_laughter_accuracy': np.float64(0.7082706766917293), 'val_laughter_precision': 0.7221420291634802, 'val_laughter_f1': 0.6992099382439322},
{'params': {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.965486380789015, 'val_gaze_accuracy': np.float64(0.7172932330827068), 'val_gaze_precision': 0.7774086640890081, 'val_gaze_f1': 0.706040209081325, 'val_laughter_accuracy': np.float64(0.6887218045112782), 'val_laughter_precision': 0.7078333442613431, 'val_laughter_f1': 0.6753586127855623},
{'params': {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.0022594862514072, 'val_gaze_accuracy': np.float64(0.7218045112781954), 'val_gaze_precision': 0.7696360944792294, 'val_gaze_f1': 0.709503414209287, 'val_laughter_accuracy': np.float64(0.7127819548872181), 'val_laughter_precision': 0.7260175132222102, 'val_laughter_f1': 0.7043455353557522},
{'params': {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.9781254728635153, 'val_gaze_accuracy': np.float64(0.7233082706766917), 'val_gaze_precision': 0.7620255457921913, 'val_gaze_f1': 0.7121341143556565, 'val_laughter_accuracy': np.float64(0.7127819548872181), 'val_laughter_precision': 0.733884650509444, 'val_laughter_f1': 0.7012197977604149},
{'params': {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.001724812719557, 'val_gaze_accuracy': np.float64(0.7218045112781954), 'val_gaze_precision': 0.7696360944792294, 'val_gaze_f1': 0.709503414209287, 'val_laughter_accuracy': np.float64(0.687218045112782), 'val_laughter_precision': 0.6926370893482501, 'val_laughter_f1': 0.6810134426976532}
]

df = pd.DataFrame(data)
df.to_csv("grid_search_results_gaze_emb_results_df_3.csv", index=False)

print(df)

                                               params  val_loss  \
0   {'bidirectional': False, 'dropout': 0.1, 'hidd...  0.987376   
1   {'bidirectional': False, 'dropout': 0.1, 'hidd...  0.995000   
2   {'bidirectional': False, 'dropout': 0.1, 'hidd...  0.989088   
3   {'bidirectional': False, 'dropout': 0.1, 'hidd...  1.006840   
4   {'bidirectional': False, 'dropout': 0.3, 'hidd...  0.992986   
5   {'bidirectional': False, 'dropout': 0.3, 'hidd...  1.021176   
6   {'bidirectional': False, 'dropout': 0.3, 'hidd...  0.974932   
7   {'bidirectional': False, 'dropout': 0.3, 'hidd...  0.980388   
8   {'bidirectional': True, 'dropout': 0.1, 'hidde...  0.990611   
9   {'bidirectional': True, 'dropout': 0.1, 'hidde...  1.000159   
10  {'bidirectional': True, 'dropout': 0.1, 'hidde...  0.986305   
11  {'bidirectional': True, 'dropout': 0.1, 'hidde...  0.994032   
12  {'bidirectional': True, 'dropout': 0.3, 'hidde...  0.965486   
13  {'bidirectional': True, 'dropout': 0.3, 'hidde...  1.00225

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=8
)

# Run the grid search
results4, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results4:
      print(res)

results_df_4 = pd.DataFrame(results4)

print("\nGrid Search Results DataFrame:")
display(results_df_4)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8710

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8826

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 0.8634

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.8792464269532098, 'val_gaze_accuracy': np.float64(0.5665859564164649), 'val_gaze_precision': 0.6876271186440678, 'val_gaze_f1': 0.5655265235168382, 'val_laughter_accuracy': np.float64(0.7033898305084746), 'val_laughter_precision': 0.7015895156883682, 'val_laughter_f1': 0.70089039347243}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.9004423949453566, 'val_gaze_accuracy': np.float64(0.5617433414043583), 'val_gaze_precision': 0.6310693901130399, 'val_gaze_f

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.879246,0.566586,0.687627,0.565527,0.703390,0.701590,0.700890
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.900442,0.561743,0.631069,0.556271,0.675545,0.676648,0.675998
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.866029,0.564165,0.661281,0.562951,0.668281,0.665523,0.664402
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.863375,0.604116,0.560416,0.567229,0.709443,0.709102,0.709256
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.898419,0.546005,0.646373,0.547401,0.673123,0.671896,0.672316
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.899228,0.618644,0.559310,0.577539,0.676755,0.677737,0.677166
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.902706,0.554479,0.648269,0.553031,0.693705,0.692299,0.692632
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.897147,0.618644,0.560856,0.578757,0.674334,0.675321,0.674748
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.869701,0.556901,0.648311,0.548163,0.668281,0.666602,0.667047
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.890636,0.607748,0.561217,0.569625,0.690073,0.689885,0.689975


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

# Run the grid search
results5, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results5:
      print(res)

results_df_5 = pd.DataFrame(results5)

print("\nGrid Search Results DataFrame:")
display(results_df_5)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.7902

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8265

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 0.7719

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.7896878023942312, 'val_gaze_accuracy': np.float64(0.5785024154589372), 'val_gaze_precision': 0.5310035892113619, 'val_gaze_f1': 0.5082382030208117, 'val_laughter_accuracy': np.float64(0.7536231884057971), 'val_laughter_precision': 0.7509796791749449, 'val_laughter_f1': 0.7516453593368745}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.805356353521347, 'val_gaze_accuracy': np.float64(0.532608695652174), 'val_gaze_precision': 0.3341080434063834, 'val_gaze_f

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.789688,0.578502,0.531004,0.508238,0.753623,0.750980,0.751645
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.805356,0.532609,0.334108,0.399702,0.759662,0.756403,0.754954
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.783124,0.567633,0.529143,0.499008,0.763285,0.760475,0.760833
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.794541,0.559179,0.431051,0.472824,0.754831,0.752285,0.752955
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.771873,0.566425,0.514188,0.498369,0.764493,0.761401,0.760965
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.792074,0.530193,0.329471,0.398577,0.748792,0.746592,0.747328
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.785105,0.585749,0.540258,0.512338,0.739130,0.735250,0.735563
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.790934,0.573671,0.526209,0.504161,0.736715,0.734580,0.735366
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.787462,0.567633,0.529143,0.499008,0.752415,0.749287,0.749752
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.804967,0.571256,0.497210,0.505277,0.739130,0.735600,0.736218


In [ ]:
results_df_4.to_csv('grid_search_results_gaze_emb_results_df_4.csv', index=False)

In [ ]:
# results_df_2.to_csv('grid_search_results_gaze_emb_results_df_2.csv', index=False)
# results_df_3.to_csv('grid_search_results_gaze_emb_results_df_3.csv', index=False)
# results_df_4.to_csv('grid_search_results_gaze_emb_results_df_4.csv', index=False)
results_df_5.to_csv('grid_search_results_gaze_emb_results_df_5.csv', index=False)

### Results - with Gaze Relation

In [ ]:
# Define the parameter grid
param_grid = {
    'hidden_dim': [128, 256],
    'num_layers': [1, 2],
    'dropout': [0.1, 0.3],
    'bidirectional': [False, True],
    'lr': [0.003],
    'weight_decay': [1e-5],
    'num_epochs': [10]
}

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

# Run the grid search
results_gr, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr:
      print(res)

results_gr_df = pd.DataFrame(results_gr)

print("\nGrid Search Results DataFrame:")
display(results_gr_df)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

Validation Loss for this combo: 1.2242

--- Running combination 2/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8764

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.0158

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 0.8764

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.2241554260253906, 'val_gaze_accuracy': np.float64(0.5186522262334536), 'val_gaze_precision': 0.2690001317769176, 'val_gaze_f1': 0.35426166324345254, 'val_laughter_accuracy': np.float64(0.4741275571600481), 'val_laughter_precision': 0.7520069495676164, 'val_laughter_f1': 0.3103333396549864}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.382271409034729, 'val_gaze_accuracy': np.float64(0.5186522262334536), 'val_gaze_precision': 0.2690001317769176, 'val_gaze_

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.224155,0.518652,0.269,0.354262,0.474128,0.752007,0.310333
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.382271,0.518652,0.269,0.354262,0.469314,0.220256,0.299808
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.026218,0.518652,0.269,0.354262,0.601685,0.728520,0.553961
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.968486,0.518652,0.269,0.354262,0.743682,0.743430,0.743313
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.262293,0.518652,0.269,0.354262,0.469314,0.220256,0.299808
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.511081,0.518652,0.269,0.354262,0.469314,0.220256,0.299808
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.003610,0.518652,0.269,0.354262,0.731649,0.734681,0.731866
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.047509,0.518652,0.269,0.354262,0.469314,0.220256,0.299808
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.082533,0.518652,0.269,0.354262,0.557160,0.625146,0.514186
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.962299,0.518652,0.269,0.354262,0.723225,0.723950,0.721528


In [ ]:
results_gr_df.to_csv('grid_search_results_gaze_emb_results_gr_df.csv', index=False)


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

# Run the grid search
results_gr2, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr2:
      print(res)

results_gr_df_2 = pd.DataFrame(results_gr2)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_2)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8298

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8957

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 0.8158

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.8422935903072357, 'val_gaze_accuracy': np.float64(0.5040322580645161), 'val_gaze_precision': 0.45488407873118175, 'val_gaze_f1': 0.4739570481924407, 'val_laughter_accuracy': np.float64(0.6854838709677419), 'val_laughter_precision': 0.6936585117918135, 'val_laughter_f1': 0.6850746644506559}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.8687033355236053, 'val_gaze_accuracy': np.float64(0.5120967741935484), 'val_gaze_precision': 0.44537812705727453, 'val_gaz

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.842294,0.504032,0.454884,0.473957,0.685484,0.693659,0.685075
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.868703,0.512097,0.445378,0.472361,0.689516,0.699158,0.688849
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.842358,0.518145,0.443363,0.473877,0.721774,0.733089,0.714555
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.830473,0.518145,0.447760,0.476370,0.679435,0.678960,0.677968
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.832989,0.518145,0.447760,0.476370,0.709677,0.711455,0.706733
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.893656,0.504032,0.450217,0.471418,0.671371,0.674814,0.671648
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.842862,0.518145,0.447760,0.476370,0.727823,0.733012,0.723608
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.883072,0.500000,0.455899,0.472638,0.689516,0.689348,0.687798
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.852276,0.504032,0.454884,0.473957,0.669355,0.676448,0.660309
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.815812,0.518145,0.443363,0.473877,0.703629,0.704196,0.701465


In [ ]:
results_gr_df_2.to_csv('grid_search_results_gaze_emb_results_gr_df_2.csv', index=False)


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.6,
    val_size=0.2,
    n_parts=8
)

# Run the grid search
results_gr3, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr3:
      print(res)

results_gr_df_3 = pd.DataFrame(results_gr3)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_3)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.0332

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.0604

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 0.9919

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.0697855552037556, 'val_gaze_accuracy': np.float64(0.4195488721804511), 'val_gaze_precision': 0.5096773788238567, 'val_gaze_f1': 0.43893191103197854, 'val_laughter_accuracy': np.float64(0.6932330827067669), 'val_laughter_precision': 0.709977252688951, 'val_laughter_f1': 0.681480198983259}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.103019376595815, 'val_gaze_accuracy': np.float64(0.4075187969924812), 'val_gaze_precision': 0.4983515882256262, 'val_gaze_f1

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.069786,0.419549,0.509677,0.438932,0.693233,0.709977,0.681480
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.103019,0.407519,0.498352,0.423191,0.670677,0.677793,0.662065
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.024306,0.412030,0.535511,0.442262,0.706767,0.727913,0.694574
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.077744,0.418045,0.493746,0.431759,0.714286,0.732260,0.704056
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.026259,0.439098,0.516054,0.457253,0.705263,0.724713,0.693591
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.122697,0.419549,0.484871,0.433809,0.672180,0.676959,0.665378
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.063699,0.419549,0.525414,0.444725,0.678195,0.687990,0.668564
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.090409,0.436090,0.512709,0.449497,0.679699,0.688775,0.670642
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.991875,0.433083,0.528882,0.453666,0.705263,0.725673,0.693204
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.127359,0.433083,0.506199,0.450257,0.669173,0.671388,0.664278


In [ ]:
results_gr_df_3.to_csv('grid_search_results_gaze_emb_results_gr_df_3.csv', index=False)


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=8
)

# Run the grid search
results_gr4, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr4:
      print(res)

results_gr_df_4 = pd.DataFrame(results_gr4)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_4)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8678

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8875

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 0.8565

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.8863336278332604, 'val_gaze_accuracy': np.float64(0.5435835351089588), 'val_gaze_precision': 0.6147112923990432, 'val_gaze_f1': 0.5432247123973788, 'val_laughter_accuracy': np.float64(0.6997578692493946), 'val_laughter_precision': 0.6981823068985114, 'val_laughter_f1': 0.6983727439953507}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.9139965507719252, 'val_gaze_accuracy': np.float64(0.6222760290556901), 'val_gaze_precision': 0.5622548213506019, 'val_gaze_

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.886334,0.543584,0.614711,0.543225,0.699758,0.698182,0.698373
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.913997,0.622276,0.562255,0.581536,0.676755,0.680184,0.677749
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.876095,0.560533,0.648699,0.548950,0.680387,0.679499,0.679843
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.884778,0.616223,0.569278,0.581870,0.694915,0.694230,0.694507
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.882598,0.559322,0.632366,0.553292,0.694915,0.693468,0.693781
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.956456,0.627119,0.564220,0.585304,0.657385,0.658658,0.657905
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.872668,0.558111,0.642693,0.552893,0.668281,0.666602,0.667047
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.907117,0.506053,0.455689,0.454298,0.673123,0.671366,0.671764
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.856515,0.554479,0.636297,0.549583,0.697337,0.695428,0.695035
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.899681,0.613801,0.558268,0.574467,0.674334,0.675321,0.674748


In [ ]:
results_gr_df_4.to_csv('grid_search_results_gaze_emb_results_gr_df_4.csv', index=False)

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

# Run the grid search
results_gr5, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr5:
      print(res)

results_gr_df_5 = pd.DataFrame(results_gr5)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_5)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.7832

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.7889

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 0.7832

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.7839748611052831, 'val_gaze_accuracy': np.float64(0.5434782608695652), 'val_gaze_precision': 0.4379429879959385, 'val_gaze_f1': 0.4245607503463926, 'val_laughter_accuracy': np.float64(0.7475845410628019), 'val_laughter_precision': 0.7454687412377042, 'val_laughter_f1': 0.7462025562634679}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.8040351470311483, 'val_gaze_accuracy': np.float64(0.5265700483091788), 'val_gaze_precision': 0.32742924280613317, 'val_gaze

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.783975,0.543478,0.437943,0.424561,0.747585,0.745469,0.746203
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.804035,0.526570,0.327429,0.392378,0.741546,0.739921,0.740574
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.786077,0.532609,0.334108,0.399702,0.759662,0.756932,0.757456
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.835351,0.532609,0.334108,0.399702,0.716184,0.718534,0.717182
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.803623,0.550725,0.446265,0.434515,0.747585,0.744335,0.744869
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.785707,0.500000,0.301930,0.356133,0.768116,0.766705,0.767244
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.795570,0.522947,0.323169,0.387865,0.742754,0.739520,0.740191
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.796129,0.531401,0.330118,0.400915,0.741546,0.740420,0.740913
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.788750,0.532609,0.426673,0.417248,0.759662,0.756932,0.757456
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.799879,0.528986,0.327632,0.398159,0.745169,0.743236,0.743951


In [ ]:
results_gr_df_5.to_csv('grid_search_results_gaze_emb_results_gr_df_5.csv', index=False)

## After Grid Search

In [ ]:
model = training_model(EnfantReactionPredictor,
               DatasetParams(isGazeRelation=False, isMixedPhases=True, train_size=0.7, val_size=0.15, n_parts=4),
               ModelParams(hidden_dim = 256, num_layers = 1, dropout = 0.1, bidirectional=True, input_size=416),
               TrainParams(lr = 0.003, weight_decay = 1e-5, num_epochs = 25)
               )

In [ ]:
# Test Loss: 1.2736, Test Gaze Acc: 0.4663, Test Gaze Prec: 0.2921, Test Gaze F1: 0.3592, Test Laughter Acc: 0.5218, Test Laughter Prec: 0.5995, Test Laughter F1: 0.5216

In [ ]:
model2 = training_model(EnfantReactionPredictor,
               DatasetParams(isGazeRelation=False, isMixedPhases=True, train_size=0.7, val_size=0.15, n_parts=4),
               ModelParams(hidden_dim = 256, num_layers = 2, dropout = 0.1, bidirectional=True, input_size=416),
               TrainParams(lr = 0.003, weight_decay = 1e-5, num_epochs = 25)
               )

Starting training...


Training Epochs:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch [1/25], 	Train Loss: 0.8099,  Train Gaze Acc: 0.3641, Train Gaze Prec: 0.5160, Train Gaze F1: 0.3311, Train Laughter Acc: 0.7399, Train Laughter Prec: 0.5474, Train Laughter F1: 0.6293, 
	Val Loss: 1.2861, Val Gaze Acc: 0.4133, Val Gaze Prec: 0.4434, Val Gaze F1: 0.4227, Val Laughter Acc: 0.5323, Val Laughter Prec: 0.2833, Val Laughter F1: 0.3698
Epoch [2/25], 	Train Loss: 0.6695,  Train Gaze Acc: 0.3259, Train Gaze Prec: 0.5085, Train Gaze F1: 0.2855, Train Laughter Acc: 0.7794, Train Laughter Prec: 0.7834, Train Laughter F1: 0.7267, 
	Val Loss: 0.9755, Val Gaze Acc: 0.3387, Val Gaze Prec: 0.4578, Val Gaze F1: 0.3829, Val Laughter Acc: 0.6331, Val Laughter Prec: 0.6986, Val Laughter F1: 0.5847
Epoch [3/25], 	Train Loss: 0.6479,  Train Gaze Acc: 0.5138, Train Gaze Prec: 0.5142, Train Gaze F1: 0.4679, Train Laughter Acc: 0.7893, Train Laughter Prec: 0.8123, Train Laughter F1: 0.7967, 
	Val Loss: 0.9281, Val Gaze Acc: 0.5040, Val Gaze Prec: 0.4549, Val Gaze F1: 0.4740, Val Laughter

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

model_params = ModelParams(
    hidden_dim=256,
    bidirectional=True,
    num_layers=1,
    dropout=0.5,
    input_size=416
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 15
)

model = training_model(EnfantReactionPredictor, dataset_params, model_params, train_params)

Starting training...


Training Epochs:   0%|          | 0/15 [00:00<?, ?it/s]

Epoch [1/15], 	Train Loss: 2.5427,  Train Gaze Acc: 0.6651, Train Gaze Prec: 0.4447, Train Gaze F1: 0.5330, Train Laughter Acc: 0.6374, Train Laughter Prec: 0.6293, Train Laughter F1: 0.5514, 
	Val Loss: 2.6428, Val Gaze Acc: 0.5187, Val Gaze Prec: 0.2690, Val Gaze F1: 0.3543, Val Laughter Acc: 0.4994, Val Laughter Prec: 0.5631, Val Laughter F1: 0.4135
Epoch [2/15], 	Train Loss: 2.0813,  Train Gaze Acc: 0.6651, Train Gaze Prec: 0.4447, Train Gaze F1: 0.5330, Train Laughter Acc: 0.6206, Train Laughter Prec: 0.5753, Train Laughter F1: 0.4806, 
	Val Loss: 2.2266, Val Gaze Acc: 0.5187, Val Gaze Prec: 0.2690, Val Gaze F1: 0.3543, Val Laughter Acc: 0.4693, Val Laughter Prec: 0.2203, Val Laughter F1: 0.2998
Epoch [3/15], 	Train Loss: 1.6974,  Train Gaze Acc: 0.6651, Train Gaze Prec: 0.4423, Train Gaze F1: 0.5313, Train Laughter Acc: 0.6224, Train Laughter Prec: 0.6570, Train Laughter F1: 0.4815, 
	Val Loss: 1.8803, Val Gaze Acc: 0.5187, Val Gaze Prec: 0.2690, Val Gaze F1: 0.3543, Val Laughter

In [ ]:
class EnfantReactionPredictor2(nn.Module):
    def __init__(self,
                 vocabs,
                 isGazeRelation,
                 hidden_dim = 16,
                 num_layers = 2,
                 dropout = 0.0,
                 input_size = 40,
                 bidirectional = False
                 ):
        super().__init__()

        self.isGazeRelation = isGazeRelation

        # Vocabs
        self.vocab_gaze = vocabs["gaze"]
        self.vocab_utterance = vocabs["utterance"]
        self.vocab_prosody = vocabs["prosody"]
        self.vocab_facial = vocabs["facial"]
        self.vocab_laughter = vocabs["laughter"]
        self.vocab_laughter_chi = vocabs["laughter_chi"]

        if self.isGazeRelation:
            self.vocab_gazerelation = vocabs["gazerelation"]

        # Embedding layers
        # self.embed_gaze = nn.Embedding(len(vocab_gaze), 8)
        self.embed_utterance = nn.Embedding(len(vocab_utterance), 8)
        self.embed_prosody = nn.Embedding(len(vocab_prosody), 4)
        self.embed_facial = nn.Embedding(len(vocab_facial), 8)
        self.embed_laughter = nn.Embedding(len(vocab_laughter), 4)

        self.sentence_encoder = SentenceTransformer('all-MiniLM-L6-v2')

        self.embed_laughter_chi = nn.Embedding(len(vocab_laughter_chi), 4)

        if self.isGazeRelation:
            self.embed_gazerelation = nn.Embedding(len(self.vocab_gazerelation), 4)

        # LSTM
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,    # so input shape is (batch, seq_len, input_dim)
            dropout=dropout,      # optional dropout between layers
            bidirectional=bidirectional
        )

        self.dropout = nn.Dropout(dropout)

        # Determine the input size for the decoder based on bidirectionality
        decoder_input_dim = hidden_dim * (2 if bidirectional else 1)

        # Decoder to predict child gaze, laughter and mother's action
        self.decoder_laughter = nn.Linear(decoder_input_dim, len(vocab_laughter_chi))
        self.project_gaze = nn.Linear(decoder_input_dim, self.sentence_encoder.get_sentence_embedding_dimension())

        if self.isGazeRelation:
            self.decoder_gazerelation = nn.Linear(decoder_input_dim, len(self.vocab_gazerelation))

    # method for finding value's index in list
    def lookup(self, vocab, value):
        if value is None:
            value = "<UNK>"
        return vocab.get(value, vocab["<UNK>"])

    # converting inputs into indices
    def encode_input_batch(self, batch_sequence_of_dicts, device):
        all_seqs = []

        for sequence in batch_sequence_of_dicts:
            seq = []

            for step_dict in sequence:
                # gaze = torch.tensor([self.lookup(self.vocab_gaze, step_dict.get("Gaze@MOT"))], dtype=torch.long)
                utterance = torch.tensor([self.lookup(self.vocab_utterance, step_dict.get("Utterance@MOT"))], dtype=torch.long)
                prosody = torch.tensor([self.lookup(self.vocab_prosody, step_dict.get("Prosody@MOT"))], dtype=torch.long)
                facial = torch.tensor([self.lookup(self.vocab_facial, step_dict.get("Facial@MOT"))], dtype=torch.long)
                laughter = torch.tensor([self.lookup(self.vocab_laughter, step_dict.get("Laughter@MOT"))], dtype=torch.long)

                action_text = step_dict.get("Action@MOT") or "<UNK>"
                encoded_action = torch.tensor(self.sentence_encoder.encode(action_text), dtype=torch.float32).unsqueeze(0)

                gaze_text = step_dict.get("Gaze@MOT") or "<UNK>"
                encoded_gaze = torch.tensor(self.sentence_encoder.encode(gaze_text), dtype=torch.float32).to(device).unsqueeze(0)

                step_emb = torch.cat([
                    encoded_gaze,
                    self.embed_utterance(utterance),
                    self.embed_prosody(prosody),
                    self.embed_facial(facial),
                    self.embed_laughter(laughter),
                    encoded_action
                ], dim=-1)  # shape: [1, D_total]

                seq.append(step_emb)

            seq = torch.cat(seq, dim=0)  # shape: [T, D_total]
            all_seqs.append(seq)

        batch_tensor = torch.stack(all_seqs).to(device)  # shape: [B, T, D_total]
        return batch_tensor


    def forward(self, batch_sequence_of_dicts, device):
        # Collect encoded steps and targets
        encoded_steps = self.encode_input_batch(batch_sequence_of_dicts, device)

        # Build the input tensor for LSTM: (batch_size, seq_len, input_dim)
        # The collate function creates sequences of length 1, so shape will be [batch_size, 1, input_dim]

        # Run the LSTM over the full sequence
        output, (h_n, c_n) = self.lstm(encoded_steps)         # output: [batch_size, 1, hidden_dim]

        # Apply decoders to the output of the LSTM for the single time step
        # Squeeze to remove the sequence length dimension of size 1
        logits_laughter = self.decoder_laughter(output.squeeze(1)) # [batch_size, num_laughter]
        predicted_gaze_embeddings = self.project_gaze(output.squeeze(1))

        if self.isGazeRelation:
            logits_gazerelation = self.decoder_gazerelation(output.squeeze(1)) # [batch_size, num_gazerelation]
            return (predicted_gaze_embeddings, logits_laughter, logits_gazerelation)

        return predicted_gaze_embeddings, logits_laughter

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

model_params = ModelParams(
    hidden_dim=256,
    bidirectional=True,
    num_layers=2,
    dropout=0.1,
    input_size=792
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 15
)

model = training_model(EnfantReactionPredictor2, dataset_params, model_params, train_params)

Starting training...


Training Epochs:   0%|          | 0/15 [00:00<?, ?it/s]

Epoch [1/15], 	Train Loss: 2.8301,  Train Gaze Acc: 0.5142, Train Gaze Prec: 0.2644, Train Gaze F1: 0.3492, Train Laughter Acc: 0.7399, Train Laughter Prec: 0.5474, Train Laughter F1: 0.6293, 
	Val Loss: 2.8793, Val Gaze Acc: 0.5363, Val Gaze Prec: 0.2876, Val Gaze F1: 0.3744, Val Laughter Acc: 0.5323, Val Laughter Prec: 0.2833, Val Laughter F1: 0.3698
Epoch [2/15], 	Train Loss: 2.3047,  Train Gaze Acc: 0.5142, Train Gaze Prec: 0.2644, Train Gaze F1: 0.3492, Train Laughter Acc: 0.7399, Train Laughter Prec: 0.5474, Train Laughter F1: 0.6293, 
	Val Loss: 2.4195, Val Gaze Acc: 0.5363, Val Gaze Prec: 0.2876, Val Gaze F1: 0.3744, Val Laughter Acc: 0.5323, Val Laughter Prec: 0.2833, Val Laughter F1: 0.3698
Epoch [3/15], 	Train Loss: 1.7017,  Train Gaze Acc: 0.5142, Train Gaze Prec: 0.2644, Train Gaze F1: 0.3492, Train Laughter Acc: 0.7399, Train Laughter Prec: 0.5474, Train Laughter F1: 0.6293, 
	Val Loss: 1.9515, Val Gaze Acc: 0.5363, Val Gaze Prec: 0.2876, Val Gaze F1: 0.3744, Val Laughter